## About the PostgreSQL relational database in the Smart Village Platform

> A SQL statement goes into a restaurant, walks up to two tables, and asks "may I join you?" One of the members at one of the tables responds, "Are we related?"

A relational SQL database like PostgreSQL is required by the ScorpioBroker to store smart device context data for each smart device. We also store configurable user data like traffic simulations, simulation reports, and more in the Smart Village application with PostgreSQL. 

A relational database like PostgreSQL is where live persistent data in an application belongs. The Smart Village Platform uses two separate tools for storing  data, including a relational database and a search engine. Some data in the Smart Village Platform is stored in the relational database as normalized data and in the search engine as denormalized data in the same database transaction. Normalized data in the relational database is the raw input data received without any additional calculations from the input data being stored. Denormalized data in the search engine is based off of the normalized data in the relational database, but with additional calculations performed to make it easily queriable with all the data required for display in the API or in a graphical dashboard. Some data may come from other sources, in which case it's not necessary to persist the data in the relational database and can be stored only in the search engine for retrieval in the API. PostgreSQL is the very best transactional database with support for IoT device data because it handles geolocation, unstructured, and structured data with confidence and ACID compliance. ACID compliance is a set of database characteristics consisting of Atomicity, Consistency, Isolation, and Durability that ensure that database transactions are completed efficiently. We can take advantage of PostgreSQL in the Smart Village Platform and ScorpioBroker at the same time. 


## Deploy PostgreSQL in NERC OpenShift

Run the command below to deploy the database create SQL scripts as a Kubernetes ConfigMap to the cloud. These scripts will be run later, when the PostgreSQL pod is running to initialize the Smart Village database. 

In [ ]:
%%bash
oc create configmap smartvillage-db-create --from-file ~/smartabyar-smartvillage/src/main/resources/sql/db-create.sql
echo DONE

If you are curious what the `db-create.sql` SQL script looks like and what it creates, run the command below. 

In [ ]:
%%bash
cat ~/smartabyar-smartvillage/src/main/resources/sql/db-create.sql
echo DONE

Here are some interesting things to note about the database schema for the Smart Village Platform. 

- Notice that the SQL script creates new database tables if they do not already exist. 
- It then creates each table column one-at-a-time if it does not already exist. 
- Each column is added to a specific table. Each column has a type and a name. 
- Some columns have additional contstraints like `created timestamp with time zone`. 
- A column with type `jsonb` can contain unstructured, nested JSON Objects and Arrays. 
- Some columns are related to other columns in other tables, like `smartTrafficLightKey bigint references SmartTrafficLight(pk)`. 
- Each table contains the same primary key column to give a unique number value to each record, `pk bigserial primary key`. 
- Each table also contains a column for optional legacy IDs for storing the previous unique IDs from a legacy application where the records came from. 


If you are curious what the EdgePostgres custom resource definition looks like that you are deploying in NERC OpenShift, run the command below. 

In [ ]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgepostgress/postgres/edgepostgres.yaml
echo DONE

Here are some useful things to note about the configuration of PostgreSQL. 

- `enable_dev_nodeports: false` It's possible to enable nodeports to directly connect to PostgreSQL from your local computer, but not in NERC OpenShift, so we set this to false. 
- `crunchy: false` We can deploy the database with the Crunchy Postgres Operator, but Crunchy requires a large amount of memory and CPU resource quota, so we deploy a simplified container of Postgres instead in NERC OpenShift. 
- The `smartvillage-db-create` configMap definition configures the `db-create.sql` SQL script we looked at earlier to be run as part of the deployment. 
- The `instances:` definition defines the memory and CPU requests and limits for our database pod. We have to keep this lower than I would recommend for a production deployment because we have limited resources available in NERC OpenShift for the many running Smart Village services. 
- The `users:` definition creates a `smartvillage` database with a a `smartvillage` user as the owner, and a `scorpiobroker` database with a `scorpiobroker` user as the owner. 
- The `pg_hba:` definition allows the database users to be trusted to connect to their given databases and the correct password, which is automatically generated during deployment. 

For more information about the EdgePostgres custom resource definition, [see the full EdgePostgres schema here](https://github.com/smartabyar-smartvillage/smartvillage-operator/blob/main/config/crd/bases/smartvillage.computate.org_edgepostgress.yaml). 

Run the Ansible Playbook below to deploy PostgreSQL to the cloud. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgepostgres.yaml \
  -e ansible_operator_meta_namespace=$(oc get project -o jsonpath={.items[0].metadata.name}) \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgepostgress/postgres/edgepostgres.yaml
echo DONE

You may see a play recap that has failed. 
This is expected because the postgres pod is just now getting created. 
The final tasks in the playbook expect the database create SQL scripts to be run for the smartvillage application in postgres. 

Retry the playbook once the postgres pod is running. 


### View PostgreSQL pod details
After running the Ansible Playbook, it will take a minute before the PostgreSQL pod is up and running. Run the command below until the PostgreSQL pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c get pod -l postgres-operator.crunchydata.com/instance-set=postgres
oc -n smart-village-faeeb6c wait pod -l postgres-operator.crunchydata.com/instance-set=postgres --for=condition=Ready --timeout=2m
oc -n smart-village-faeeb6c get pod -l postgres-operator.crunchydata.com/instance-set=postgres
echo DONE

Now re-run the ansible Playbook to complete the last tasks for PostgreSQL. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-edgepostgres.yaml \
  -e ansible_operator_meta_namespace=$(oc get project -o jsonpath={.items[0].metadata.name}) \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/edgepostgress/postgres/edgepostgres.yaml
echo DONE

### Review the Ansible Playbook for PostgreSQL
After running the Ansible Playbook, you should see a PLAY RECAP where no tasks have failed `failed=0`. 

### View PostgreSQL pod logs
If your PostgreSQL pod does not reach the STATUS Running, you can run the command below to view the pod logs of PostgreSQL and check for other errors that may have occured. 

In [ ]:
%%bash
oc -n smart-village-faeeb6c logs -l app=postgres


## Next...
I hope that answers your questions about PostgreSQL in the Smart Village Platform. 
- If you have additional questions or issues, please [create an issue for the course here](https://github.com/smartabyar-smartvillage/smartabyar-smartvillage-nerc-course/issues). 
- Otherwise, please continue to the next notebook [06-about-scorpiobroker.ipynb](06-about-scorpiobroker.ipynb). 